# Exam
1. Load the dataset 
https://huggingface.co/datasets/poem_sentiment
2. Set 'bert-base-uncased' tokenizer and model for your task.
3. Tokenize data and set train/test/validation data
4. Create Trainer with the correct configuration
5. Use Trainer and fine-tune your model for 5 epochs.
6. Predict on test data and report accuracy.


In [1]:
# IMPORTANT
!pip install -q transformers==4.28.0
!pip install -q datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.0/7.0 MB 40.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 4.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 75.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 474.6/474.6 kB 21.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 110.5/110.5 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.5/212.5 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.3/134.3 kB 13.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.5/114.5 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 149.6/149.6 kB 20.3 MB/s eta 0:00:00


In [2]:
from datasets import load_dataset, DatasetDict

# YOUR CODE HERE - 1
# Download the dataset
from datasets import load_dataset, DatasetDict


imdb_dataset = load_dataset("poem_sentiment")

Generating train split:   0%|          | 0/892 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/105 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/104 [00:00<?, ? examples/s]

Dataset poem_sentiment downloaded and prepared to /root/.cache/huggingface/datasets/poem_sentiment/default/1.0.0/4e44428256d42cdde0be6b3db1baa587195e91847adabf976e4f9454f6a82099. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [4]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# YOUR CODE HERE - 2
# Load the model and tokenizer
from transformers import BertModel, BertTokenizerFast
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")

bert = BertModel.from_pretrained("bert-base-uncased")
embedding_matrix = bert.embeddings.word_embeddings.weight

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
#['verse_text']
# YOUR CODE HERE - 3
import warnings
from datasets import load_dataset, DatasetDict
warnings.filterwarnings("ignore")
from torch.utils.data import DataLoader
# Set train/test/validation
train_data = None
val_data = None
test_data = None

def truncate(example):
    return {
        'verse_text': " ".join(example['verse_text'].split()[:50]),
        'label': example['label']
    }

# Take 128 random examples for train and 32 validation
small_imdb_dataset = DatasetDict(
    train_data=imdb_dataset['train'].shuffle(seed=1111).map(truncate),
    test_data=imdb_dataset['train'].shuffle(seed=1111).map(truncate),
    val_data=imdb_dataset['validation'].shuffle(seed=1111).map(truncate)
)

Map:   0%|          | 0/892 [00:00<?, ? examples/s]

Map:   0%|          | 0/105 [00:00<?, ? examples/s]

In [13]:
from transformers import TrainingArguments, Trainer
import numpy as np

# You can optionally use this function with the right parameter to compute results.
# You don't have to use this though. You can do the caculation anyway you like.
def compute_metrics(eval_pred):
    """Called at the end of validation. Gives accuracy"""
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    # calculates the accuracy
    return {"accuracy": np.mean(predictions == labels)}

# YOUR CODE HERE - 4
# create trainer

arguments = TrainingArguments(
    output_dir="sample_hf_trainer",
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    evaluation_strategy="epoch", # run validation at the end of each epoch
    save_strategy="epoch",
    learning_rate=2e-5,
    load_best_model_at_end=True,
    seed=224
)

trainer = Trainer(
    model=bert,
    args=arguments,
    train_dataset=train_data,
    eval_dataset=val_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

In [15]:
# YOUR CODE HERE - 5
# Train
trainer.train(train_data)

ValueError: ignored

In [ ]:
# YOUR CODE HERE - 6
# Evalute Test data and report results